In [47]:
import numpy
import plotly.graph_objects as go

In [48]:
V_coll = numpy.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
])

V_fem = numpy.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0.25, 0, 0],
    [0.5, 0, 0],
    [0.75, 0, 0],
    [0.75, 0.25, 0],
    [0.5, 0.5, 0],
    [0.25, 0.75, 0],
    [0, 0.25, 0],
    [0, 0.5, 0],
    [0, 0.75, 0],
])

In [49]:
Winv = numpy.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [0.5, 0.5, 0],
    [0, 0.5, 0.5],
    [0.5, 0, 0.5],
])

In [50]:
V_mid = Winv @ V_coll
print(V_mid)

[[0.  0.  0. ]
 [1.  0.  0. ]
 [0.  1.  0. ]
 [0.5 0.  0. ]
 [0.5 0.5 0. ]
 [0.  0.5 0. ]]


In [87]:
def pseudoinverse(A):
    U, Σ, Vᵀ = numpy.linalg.svd(A, full_matrices=False)
    Σ[Σ != 0] = 1 / Σ
    return Vᵀ.T @ numpy.diag(Σ) @ U.T
W = pseudoinverse(Winv)
print(W)

[[ 0.7 -0.1 -0.1  0.3 -0.1  0.3]
 [-0.1  0.7 -0.1  0.3  0.3 -0.1]
 [-0.1 -0.1  0.7 -0.1  0.3  0.3]]


In [52]:
numpy.linalg.norm(V_coll - W @ V_mid)

6.550918560462233e-16

In [53]:
# Map from V_fem to V_mid
T = numpy.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
], dtype=float)

In [54]:
numpy.linalg.norm(V_mid - T @ V_fem)

0.0

In [55]:
# Map from V_fem to V_coll through V_mid
A = W @ T
print(A)

[[ 0.7 -0.1 -0.1  0.   0.3  0.   0.  -0.1  0.   0.   0.3  0. ]
 [-0.1  0.7 -0.1  0.   0.3  0.   0.   0.3  0.   0.  -0.1  0. ]
 [-0.1 -0.1  0.7  0.  -0.1  0.   0.   0.3  0.   0.   0.3  0. ]]


In [56]:
numpy.linalg.norm(V_coll - A @ V_fem)

6.550918560462233e-16

In [57]:
U = numpy.array([
    [0, 0, 0],
    [0, 0.5, 0],
    [0, 0, 0],
])
print(A.T @ U)

[[ 0.   -0.05  0.  ]
 [ 0.    0.35  0.  ]
 [ 0.   -0.05  0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.15  0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.15  0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.   -0.05  0.  ]
 [ 0.    0.    0.  ]]


In [58]:
X = V_fem + A.T @ U

In [59]:
go.Figure(data=[
    go.Scatter(x=V_fem[:, 0], y=V_fem[:, 1], mode='markers'),
    go.Scatter(x=V_mid[:, 0], y=V_mid[:, 1], mode='markers'),
    go.Scatter(x=V_coll[:, 0], y=V_coll[:, 1], mode='markers'),
])

In [78]:
X_coll = V_coll + U
X_mid = V_mid + W.T @ U
X_fem = V_fem + T.T @ W.T @ U

coll_row_order = numpy.array([0, 1, 2, 0])
mid_row_order = numpy.array([0, 3, 1, 4, 2, 5, 0])
fem_row_order = numpy.array([0, 3, 4, 5, 1, 6, 7, 8, 2, 9, 10, 11, 0])

from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=V_mid[mid_row_order, 0], y=V_mid[mid_row_order, 1], name="V_mid"), row=1, col=2),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_mid[mid_row_order, 0], y=X_mid[mid_row_order, 1], name="Deformed V_mid"), row=2, col=2),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800)

In [85]:
W_full_inv = V_coll_to_V_fem = numpy.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [0.75, 0.25, 0],
    [0.5, 0.5, 0],
    [0.25, 0.75, 0],
    [0, 0.75, 0.25],
    [0, 0.5, 0.5],
    [0, 0.25, 0.75],
    [0.75, 0, 0.25],
    [0.5, 0, 0.5],
    [0.25, 0, 0.75],
])

In [93]:
W_full = pseudoinverse(W_full_inv)
numpy.linalg.norm(V_coll - W_full @ V_fem)

3.4829515123253013e-16

In [94]:
X_coll = V_coll + U
X_fem = V_fem + W_full.T @ U

coll_row_order = numpy.array([0, 1, 2, 0])
mid_row_order = numpy.array([0, 3, 1, 4, 2, 5, 0])
fem_row_order = numpy.array([0, 3, 4, 5, 1, 6, 7, 8, 2, 9, 10, 11, 0])

from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800)